# __Diffusion MRI data analysis notebook__
#### __Last updated on:__ 19/11/2019
#### __Author:__ Rakshit Dadarwal

### __Requirements:__
#### 1. DIPY (https://dipy.org/)
#### 2. nipype (https://nipype.readthedocs.io/en/latest/)
#### 3. FSL (https://fsl.fmrib.ox.ac.uk/fsl/fslwiki)

### __This script includes:__
#### 1. Denoising
#### 2. TOPUP: Susceptibility-induced distortion correction  
#### 3. EDDY: eddy current-induced distortion and motion correction

### __Denoising__

In [1]:
import os
import nibabel as nib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import timeit; timeit.timeit()

#--------------------------------------------
#        DWI data files 
#--------------------------------------------
data_path = "/home/User/Diffusion"
dwi_file = 'dMRI.nii.gz' # 4D diffusion data file name
bvals_file = 'dMRI.bvals' # bval file name
bvecs_file = 'dMRI.bvecs' # bvec file name
output_file = 'dwi_denoised.nii.gz' # denoised output file name 

if os.path.exists(data_path):
        
    # Change directory
    cwdir = os.getcwd()
    os.chdir(data_path)
        
    #------------------------------------------------
    #         DIPY MP-PCA denoising
    #------------------------------------------------
    from dipy.denoise.localpca import mppca           
            
    fbval = os.path.join(data_path,bvals_file)
    fbvec = os.path.join(data_path,bvecs_file)
            
    from dipy.io import read_bvals_bvecs
    from dipy.core.gradients import gradient_table
            
    bvals, bvecs = read_bvals_bvecs(fbval,fbvec)
    gtab = gradient_table(bvals, bvecs)
        
    img = nib.load(os.path.join(data_path,dwi_file))
    data = img.get_data()    
            
    denoised = mppca(data, patch_radius=2)
        
    nib.save(nib.Nifti1Image(data, img.affine), os.path.join(data_path,output_file))

    print('Processing completed')
    print('Elapsed time:',timeit.timeit())
    print('Enjoy!!')

Elapsed time: 0.015462636947631836


### __TOPUP__
* Susceptibility-induced distortion correction

In [5]:
import os
import nibabel as nib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import timeit; timeit.timeit()

import nipype.interfaces.fsl as fsl

data_path = "/home/User/Diffusion"

if os.path.exists(data_path):
        
    # Change directory
    cwdir = os.getcwd()
    os.chdir(data_path)
        
    #------------------------------------------------
    #             Extract b0 image
    #------------------------------------------------
    fslroi = fsl.ExtractROI(in_file='dwi_denoised.nii.gz', 
                            roi_file='epi_b0.nii.gz', t_min=0, t_size=1)
    fslroi.run()
            
    #------------------------------------------------
    #             FSL TOPUP 
    #------------------------------------------------
    # data prep for topup
    merger = fsl.Merge(in_files=['epi_b0.nii.gz','epi_rev.nii.gz'],
                       dimension = 't', output_type='NIFTI_GZ')
    merger.run()
        
    file = open('topup_encoding.txt','w')
    file.write('0 1 0 0.05\n0 -1 0 0.05')
    file.close()
        
    topup = fsl.TOPUP(in_file = 'epi_b0_merged.nii.gz',
                      encoding_file = 'topup_encoding.txt',
                      output_type = "NIFTI_GZ")
    topup.run()
        
    #------------------------------------------------
    #            FSL ApplyTOPUP 
    #------------------------------------------------
    applytopup = fsl.ApplyTOPUP(in_files = ['epi_b0.nii.gz', 'epi_rev.nii.gz'],
                                encoding_file = 'topup_encoding.txt',
                                in_topup_fieldcoef = "epi_b0_merged_base_fieldcoef.nii.gz",
                                in_topup_movpar = 'epi_b0_merged_base_movpar.txt',
                                output_type = "NIFTI_GZ") 
    applytopup.run()
        
    print('Processing completed')
    print('Elapsed time:',timeit.timeit())
    print('Enjoy!!')

Processing completed
Elapsed time: 0.008853044360876083
Enjoy!!


### __EDDY__
* eddy current-induced distortion and motion correction

In [ ]:
import os
import nibabel as nib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import timeit; timeit.timeit()

import nipype.interfaces.fsl as fsl

data_path = "/home/User/Diffusion"

if os.path.exists(data_path):
        
    # Change directory
    cwdir = os.getcwd()
    os.chdir(data_path)
        
    #------------------------------------------------
    #            FSL Eddy
    #------------------------------------------------
    # data prep for eddy
    btr = fsl.BET(in_file='epi_b0_corrected.nii.gz',
                     frac=0.5, out_file='brain.nii.gz', mask=True)
    btr.run()
        
    file = open('index.txt','w')
    for i in range(1, 62):
        file.write('1 ')
    file.close()
            
    eddy = fsl.Eddy(in_file = 'dwi_denoised.nii.gz',
                    in_mask  = 'brain_mask.nii.gz',
                    in_index = 'index.txt',
                    in_acqp  = 'topup_encoding.txt',
                    in_bvec  = 'dMRI.bvecs',
                    in_bval  = 'dMRI.bvals', use_cuda = False)
    eddy.run()
        
    print('Processing completed')
    print('Elapsed time:',timeit.timeit())
    print('Enjoy!!')